# Learning Langchain


## Definition of langchain

Langchain is a framework design for building apps using LLMs , it is model agnostic which means that you can you use it with any LLMs
it has a modular architecture and is based on conponents and chains

---

- Conponents ou modules are reusable elements that executer specific taks ,like data-entry , accessing external data , managing workflows and memory integration
- Chains are sequences of conponents or modules that work together to complete a goal like summarise a doc, generate an article , or provide recommandations.

---

chains are very important in the way langchain manage workflows .

---

In langchain ecosystem there is serveral other frameworks :

- LangGraph : it is base on `langchain Core`
- Langsmith is a platforme built by Langchain . it main purpose is to help devs with observability of their LLM apps . Langsmith is integrated in the langchain ecosystem but can be use outside of it


## setting up langsmith for obervability


In [2]:
!uv add dotenv

Resolved 79 packages in 2.72s                                        
⠹ Preparing packages... (0/2)                                                   ⠋ Resolving dependencies...                                                     
⠹ Preparing packages... (0/2)-------     0 B/19.78 KiB                  
⠹ Preparing packages... (0/2)2m----- 16.00 KiB/19.78 KiB                
⠹ Preparing packages... (0/2)--- 19.78 KiB/19.78 KiB                
dotenv     ------------------------------     0 B/1.85 KiB
⠹ Preparing packages... (0/2)--- 19.78 KiB/19.78 KiB                
⠹ Preparing packages... (0/2)----     0 B/1.85 KiB                      
Prepared 2 packages in 306ms                                                 
Installed 2 packages in 5ms                                 
 + dotenv==0.9.9
 + python-dotenv==1.1.0


In [71]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [5]:
# load environment variables
load_dotenv()
# get the environment variables
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING")
LANGSMITH_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

## channing 1


In [59]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.5,
    max_output_tokens=1000,
)

In [60]:
llm.invoke("Hello, how are you?")

AIMessage(content='I am doing well, thank you for asking!  How are you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--3394961c-6aa3-4f29-8287-f93aa0735350-0', usage_metadata={'input_tokens': 6, 'output_tokens': 17, 'total_tokens': 23, 'input_token_details': {'cache_read': 0}})

In [64]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an expert in marketing and promoting cultural evenent like concerts, "
    "exhibitions, and festivals. You are a marketing expert with 10 years of experience in the field."
    " You are very creative and you have a lot of ideas."
    " You are very good at writing and you can write very well. "
    "You have 10 years of experience, are very creative, and excellent at writing viral content."
)
user_prompt = HumanMessagePromptTemplate.from_template(
    """
I am organizing a concert in Paris. 
Your role is to write a social media viral type post to promote the event.
the post have to be very catchy and engaging. and less than 100 words.
The artist is {artist} and the date is {date}. 
The artist is known for {reputation}.
The concert will be held at {venue} and the ticket price is {ticket_price}.
"""
)

In [65]:
user_prompt.format(
    artist="Taylor Swift",
    date="2023-10-01",
    reputation="a pop star with a huge fan base ",
    venue="Accor Arena",
    ticket_price="$100",
)

HumanMessage(content='\nI am organizing a concert in Paris. \nYour role is to write a social media viral type post to promote the event.\nthe post have to be very catchy and engaging. and less than 100 words.\nThe artist is Taylor Swift and the date is 2023-10-01. \nThe artist is known for a pop star with a huge fan base .\nThe concert will be held at Accor Arena and the ticket price is $100.\n', additional_kwargs={}, response_metadata={})

In [66]:
first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

In [67]:
print(
    first_prompt.format(
        artist="Taylor Swift",
        date="2023-10-01",
        reputation="a pop star with a huge fan base ",
        venue="Accor Arena",
        ticket_price="$100",
    )
)


System: You are an expert in marketing and promoting cultural evenent like concerts, exhibitions, and festivals. You are a marketing expert with 10 years of experience in the field. You are very creative and you have a lot of ideas. You are very good at writing and you can write very well. You have 10 years of experience, are very creative, and excellent at writing viral content.
Human: 
I am organizing a concert in Paris. 
Your role is to write a social media viral type post to promote the event.
the post have to be very catchy and engaging. and less than 100 words.
The artist is Taylor Swift and the date is 2023-10-01. 
The artist is known for a pop star with a huge fan base .
The concert will be held at Accor Arena and the ticket price is $100.



In [68]:
chain = first_prompt | llm | StrOutputParser()


In [69]:
post = chain.invoke(
    {
        "artist": "Kendrick Lamar",
        "date": "2023-10-01",
        "reputation": "Kendrick Lamar is a Grammy-winning rapper known for his thought-provoking lyrics and powerful performances. He is known for songs like 'Not Like Us', 'DNA', 'HUMBLE', and many more.",
        "venue": "Accor Arena",
        "ticket_price": "$100",
    }
)


In [70]:
print(post)


**Option 1 (Focus on urgency & FOMO):**

🔥 PARIS! 🔥 Kendrick Lamar LIVE at Accor Arena, October 1st!  His legendary rhymes are coming to town.  Tickets are FLYING – snag yours NOW before they're GONE!  $100.  [Link to tickets] #KendrickLamarParis #AccorArena #ParisConcert #HipHop #SoldOutSoon


**Option 2 (Focus on experience):**

Experience Kendrick Lamar's UNFORGETTABLE energy LIVE in Paris!  Accor Arena, October 1st.  Witness the Grammy-winning legend perform 'HUMBLE,' 'DNA,' and more.  Tickets: $100. [Link to tickets] #KendrickLamar #Paris #Concert #HipHop #MustSee


**Option 3 (More playful & meme-able):**

My therapist told me to embrace my obsessions... so I bought tickets to see Kendrick Lamar in PARIS!  October 1st, Accor Arena.  Join me? 😉 $100. [Link to tickets] #KendrickLamar #Paris #Concert #Therapy #HipHop


**Option 4 (Short & sweet):**

Kendrick Lamar. Paris. October 1st. Accor Arena.  $100.  Need we say more?  GET TICKETS! [Link to tickets] #KendrickLamarParis #Concert

# chainning 2


In [75]:
prompt = PromptTemplate.from_template(
    " what is a good company name for a company that makes {product}?"
)
chain = prompt | llm | StrOutputParser()
company_name = chain.invoke({"product": "flying cars"})
print(company_name)

The best name will depend on the brand image you want to project (luxury, futuristic, practical, etc.). Here are some ideas, categorized for easier selection:

**Futuristic & Tech-Focused:**

* AeroDrive
* SkyDrive Dynamics
* Zenith Aviation
* Aether Motors
* IonFlight
* Quantum Ascent
* NovaFlight
* SkySpark Technologies
* Vertical Dynamics
* Apex Aeronautics

**Elegant & Sophisticated:**

* Aerion Motors
* Celestial Flight
* Nimbus Automotive
* Elysian Skies
* Sovereign Aero
* Paragon Aviation
* Zenith Aero
* Skybourne Motors

**Practical & Approachable:**

* Skyway Motors
* Air Mobility Solutions
* Urban Air
* Commuter Skies
* SkyRunner
* AeroTransit
* FlightLine
* AirVoyager

**Bold & Adventurous:**

* Skybound
* Apex Flight
* Renegade Aero
* Maverick Aviation
* Pathfinder Aero
* Horizon Chasers


**Tips for Choosing:**

* **Check for availability:** Make sure the name isn't already taken and that the domain name is available.
* **Keep it short and memorable:**  Easy to recall and 

In [79]:
description_prompt = PromptTemplate.from_template(
    "You are a world-class branding expert. Given a product description, come up with:\n"
    "- A **brand name** that is unique, catchy, and memorable.\n"
    "- A **slogan** that is creative, engaging, and aligned with the product.\n\n"
    "Product description: {product_description}\n\n"
    "Return the result in the format:\nBrand Name: <name>\nSlogan: <slogan>"
)

chain = description_prompt | llm | StrOutputParser()

result = chain.invoke(
    {
        "product_description": "a flying car that is self-driving and can be summoned via an app.",
    }
)

print(result)


Brand Name: AetherWing
Slogan: Your Sky. Your Ride. Your App.


In [83]:
get_name_prompt = PromptTemplate.from_template(
    "You are a world-class branding expert. Given a product description, come up with:\n"
    "- A **brand name** that is unique, catchy, and memorable.\n"
    "Product description: {product_description}\n\n"
    "Return the result in the format:\nBrand Name: <name>"
)
get_slogan_prompt = PromptTemplate.from_template(
    "You are a world-class branding expert. Given a product name, come up with:\n"
    "- A **slogan** that is creative, engaging, and aligned with the product.\n\n"
    "Product description: {product_name}\n\n"
    "Return the result in the format:\nSlogan: <slogan>"
)
chain_brand = get_name_prompt | llm | get_slogan_prompt | llm | StrOutputParser()

brand = chain_brand.invoke(
    {
        "product_description": "a flying car that is self-driving and can be summoned via an app.",
    }
)
print(brand)

Slogan: SkyLeap: Reach New Heights.


In [84]:
from langchain.schema.runnable import RunnableLambda, RunnableMap


get_name_prompt = PromptTemplate.from_template(
    "You are a world-class branding expert. Given a product description, come up with:\n"
    "- A **brand name** that is unique, catchy, and memorable.\n"
    "Product description: {product_description}\n\n"
    "Return the result in the format:\nBrand Name: <name>"
)
get_name_chain = get_name_prompt | llm | StrOutputParser()


extract_brand_name = RunnableLambda(
    lambda text: {"brand_name": text.split(":")[-1].strip()}
)

get_slogan_prompt = PromptTemplate.from_template(
    "You are a world-class branding expert. Given a product name, come up with:\n"
    "- A **slogan** that is creative, engaging, and aligned with the product.\n\n"
    "Product name: {brand_name}\n\n"
    "Return the result in the format:\nSlogan: <slogan>"
)
get_slogan_chain = get_slogan_prompt | llm | StrOutputParser()

extract_slogan = RunnableLambda(lambda text: {"slogan": text.split(":")[-1].strip()})


full_chain = (
    get_name_chain
    | extract_brand_name
    | RunnableMap(
        {
            "brand_name": RunnableLambda(lambda x: x["brand_name"]),
            "slogan": (
                RunnableLambda(lambda x: {"brand_name": x["brand_name"]})
                | get_slogan_chain
                | extract_slogan
            ),
        }
    )
)

result = full_chain.invoke(
    {
        "product_description": "a flying car that is self-driving and can be summoned via an app.",
    }
)

print(result)


{'brand_name': 'SkyCall', 'slogan': {'slogan': 'Connect Beyond Limits.'}}


In [99]:
# Prompts
get_name_prompt = PromptTemplate.from_template(
    "Create a catchy brand name for the following product: {product_description}\nFormat: Brand Name: <name>"
)

get_slogan_prompt = PromptTemplate.from_template(
    "Write a slogan for the brand: {brand_name}\nFormat: Slogan: <slogan>"
)

get_post_prompt = PromptTemplate.from_template(
    "You are a social media marketing expert. Write a viral post to promote a product with:\n"
    "- Brand Name: {brand_name}\n"
    "- Slogan: {slogan}\n"
    "Be creative, energetic, and hook the reader!"
)

# Sous-chaîne: nom de marque
get_name_chain = get_name_prompt | llm | StrOutputParser()
extract_brand_name = RunnableLambda(lambda x: {"brand_name": x.split(":")[-1].strip()})

# Sous-chaîne: slogan
get_slogan_chain = get_slogan_prompt | llm | StrOutputParser()
extract_slogan = RunnableLambda(lambda x: {"slogan": x.split(":")[-1].strip()})

# Sous-chaîne: post viral
get_post_chain = get_post_prompt | llm | StrOutputParser()

full_chain = (
    get_name_chain
    | extract_brand_name
    | RunnableMap(
        {
            "brand_name": RunnableLambda(lambda x: x["brand_name"]),
            "slogan": (
                RunnableLambda(lambda x: {"brand_name": x["brand_name"]})
                | get_slogan_chain
                | extract_slogan
            ),
        }
    )
    | RunnableMap(
        {
            "brand_name": RunnableLambda(lambda x: x["brand_name"]),
            "slogan": RunnableLambda(lambda x: x["slogan"]),
            "post": (
                RunnableLambda(
                    lambda x: {
                        "brand_name": x["brand_name"],
                        "slogan": x["slogan"],
                    }
                )
                | get_post_chain
            ),
        }
    )
)
result = full_chain.invoke(
    {
        "product_description": "a flying car that is self-driving and can be summoned via an app.",
    }
)
print(result)

{'brand_name': 'SkyCall', 'slogan': {'slogan': 'Connecting you to what matters, high and clear.'}, 'post': '##  🤯  Is YOUR phone call quality giving you a headache?  🤯\n\nLet\'s be honest, dropped calls, static, and that soul-crushing "you\'ve been disconnected" message are the WORST.  😫\n\nBut what if I told you there\'s a solution?  A crystal-clear, high-flying, unbelievably AMAZING solution? ✨\n\nIntroducing **SkyCall!** 🚀\n\n**(Image: A stunning, high-resolution image or short video showing someone happily using a SkyCall device in a beautiful, high-altitude setting – maybe a mountaintop, hot air balloon, or even space!  Emphasis on clear skies and a strong signal.)**\n\nSkyCall isn\'t just another phone service.  It\'s a **revolution**.  We\'re talking:\n\n* **Unbreakable Connection:** Say goodbye to dropped calls FOREVER!\n* **Crystal-Clear Audio:** Hear every word, every nuance, like you\'re in the same room.\n* **Blazing-Fast Speeds:** Download and stream without the lag.\n* **